In [0]:
FIELD = 'dra'
ISOCHRONES_PATH = '/datascope/subaru/data/cmdfit/isochrones/dartmouth/import/afep0_cfht_sdss_hsc_nb_bosz'
OBS_PATH = '/datascope/subaru/data/targeting/dSph/draco/draco_tpall3e_g24.cat'

In [0]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Ellipse, Circle
from scipy.special import logsumexp
from scipy.interpolate import interp1d

In [0]:
plt.rc('font', size=6) #controls default text size

In [0]:
%load_ext autoreload

In [0]:
%autoreload 2

In [0]:
# import debugpy
# debugpy.listen(('0.0.0.0', 5698))

# Load isochrones

In [0]:
from pfs.ga.isochrones.isogrid import IsoGrid

In [0]:
iso = IsoGrid()
iso.load(os.path.join(ISOCHRONES_PATH, 'isochrones.h5'))

In [0]:
iso.values.keys()

# Load observations

In [0]:
from pfs.ga.targeting.targets.dsph import GALAXIES
field = GALAXIES[FIELD]

In [0]:
from pfs.ga.targeting.instrument import *
from pfs.ga.targeting.diagram import CMD, CCD, ColorAxis, MagnitudeAxis
from pfs.ga.targeting.photometry import Photometry, Magnitude, Color

In [0]:
obs = field.get_text_observation_reader().read(OBS_PATH)
obs.data.head()

In [0]:
hsc = field.get_photometry(SubaruHSC)
cmd = field.get_cmd(SubaruHSC)
ccd = field.get_ccd(SubaruHSC)

In [0]:
f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_observation(axs[0], obs)
ccd.plot_observation(axs[1], obs)

f.tight_layout()

# Color cuts

In [0]:
from pfs.ga.targeting import ProbabilityMap
from pfs.ga.targeting.selection import ProbabilityCut, ProbabilitySampling, MagnitudeSelection, ColorSelection, LinearSelection

### Broadband color cut

In [0]:
mask = field.get_selection_mask(obs, nb=False, blue=False)

f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_observation(axs[0], obs)
ccd.plot_observation(axs[1], obs)

cmd.plot_observation(axs[0], obs, mask=mask, color='r')
ccd.plot_observation(axs[1], obs, mask=mask, color='r')

for ax in axs.ravel():
    ax.grid(True)

f.tight_layout()

In [0]:
mask = field.get_selection_mask(obs, nb=False, blue=False)

f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_observation(axs[0], obs, mask=mask)
ccd.plot_observation(axs[1], obs, mask=mask)

for ax in axs.ravel():
    ax.grid(True)

f.tight_layout()

### Narrow band color cuts

In [0]:
mask = field.get_nb_selection_mask(obs)
#mask = field.get_selection_mask(obs, nb=True)

f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_observation(axs[0], obs)
ccd.plot_observation(axs[1], obs)

cmd.plot_observation(axs[0], obs, mask=mask, color='r', size=1)
ccd.plot_observation(axs[1], obs, mask=mask, color='r')

for ax in axs.ravel():
    ax.grid(True)

f.tight_layout()

In [0]:
mask = field.get_selection_mask(obs, nb=False)
mask_nb = field.get_selection_mask(obs, nb=True)

f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_observation(axs[0], obs, mask=mask)
ccd.plot_observation(axs[1], obs, mask=mask)

cmd.plot_observation(axs[0], obs, mask=mask_nb, color='r', size=1)
ccd.plot_observation(axs[1], obs, mask=mask_nb, color='r')

for ax in axs.ravel():
    ax.grid(True)

f.tight_layout()

In [0]:
from pfs.ga.targeting import Isochrone
from pfs.ga.targeting.selection import IsochroneSelection

In [0]:
import pfs.ga.isochrones.tensorlib as tt

In [0]:
mask = field.get_selection_mask(obs, nb=False)
mask_nb = field.get_selection_mask(obs, nb=True)

f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_observation(axs[0], obs, mask=mask_nb)
ccd.plot_observation(axs[1], obs, mask=mask_nb)

N = 10000

t = tt.tensor(N * [10], dtype=tt.float64)
Fe_H = tt.tensor(N * [-2.5], dtype=tt.float64)
M = tt.linspace(0.6, 1.0, N, dtype=tt.float64)

mi_EEP, mi_M_ini, (hsc_g, hsc_i, hsc_nb515), mask = \
    iso.interp3d(Fe_H, t, M,
                  [iso.values['hsc_g'], iso.values['hsc_i'], iso.values['hsc_nb515']])

axs[0].plot(hsc_g - hsc_i, hsc_g + 19.557, '-b')
axs[1].plot(hsc_g - hsc_i, hsc_g - hsc_nb515, '-b')

###

t = tt.tensor(N * [10.12], dtype=tt.float64)
Fe_H = tt.tensor(N * [-1.8], dtype=tt.float64)
M = tt.linspace(0.6, 1.0, N, dtype=tt.float64)

mi_EEP, mi_M_ini, (hsc_g, hsc_i, hsc_nb515), mask = \
    iso.interp3d(Fe_H, t, M,
                  [iso.values['hsc_g'], iso.values['hsc_i'], iso.values['hsc_nb515']])

axs[0].plot(hsc_g - hsc_i, hsc_g + 19.557, '-r')
axs[1].plot(hsc_g - hsc_i, hsc_g - hsc_nb515, '-r')

for ax in axs.ravel():
    ax.grid(True)

f.tight_layout()

### Select using isochrones

In [0]:
iso_blue = Isochrone()
iso_blue.from_isogrid(hsc, iso, Fe_H=-2.5, log_t=10.135, DM=19.2)

iso_red = Isochrone()
iso_red.from_isogrid(hsc, iso, Fe_H=-1.6, log_t=10.11, DM=19.2)

In [0]:
mask = IsochroneSelection(iso_blue, cmd.axes, selection_axis=0, selection_direction='+',
                          DM=19.2, error_sigma=[-2, 0]).apply(obs)

mask &= IsochroneSelection(iso_red, cmd.axes, selection_axis=0, selection_direction='-',
                          DM=19.2, error_sigma=[2, 0]).apply(obs)

f, axs = plt.subplots(1, 2, figsize=(6, 4), dpi=120)

cmd.plot_observation(axs[0], obs, c='gray')
ccd.plot_observation(axs[1], obs, c='gray')

cmd.plot_observation(axs[0], obs, mask=mask, c='g')
ccd.plot_observation(axs[1], obs, mask=mask, c='g')

# cmd.plot_isochrone(axs[0], iso_blue, observed=True, error_sigma=[-2.0, -2.0], c='blue', lw=0.5)
# cmd.plot_isochrone(axs[0], iso_red, observed=True, error_sigma=[2.0, 2.0], c='red', lw=0.5)
cmd.plot_isochrone(axs[0], iso_blue, observed=True, c='blue', lw=0.5)
cmd.plot_isochrone(axs[0], iso_red, observed=True, c='red', lw=0.5)

f.tight_layout()